In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .getOrCreate() 

23/05/06 17:56:45 WARN Utils: Your hostname, wedivv-H110M-S2V resolves to a loopback address: 127.0.1.1; using 192.168.1.44 instead (on interface wlp5s0)
23/05/06 17:56:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/06 17:56:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.read.csv('./data/Extrato.csv', header=True, sep=';').show(5)

+------------------------+
| Extrato Conta Corrente |
+------------------------+
|                  Conta |
|                Período |
|         Data Lançamento|
|              17/03/2023|
|              13/03/2023|
+------------------------+
only showing top 5 rows



In [3]:
from pyspark.sql.functions import monotonically_increasing_id

df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("sep", ";")
    .load("./data/Extrato.csv")
)

df_with_id = df.withColumn("id", monotonically_increasing_id())

df_with_id.show(5)

+------------------------+---+
| Extrato Conta Corrente | id|
+------------------------+---+
|                  Conta |  0|
|                Período |  1|
|         Data Lançamento|  2|
|              17/03/2023|  3|
|              13/03/2023|  4|
+------------------------+---+
only showing top 5 rows



In [4]:
df_filtered = df_with_id.filter(df_with_id.id >= 3)
df_filtered.show(5)

+------------------------+---+
| Extrato Conta Corrente | id|
+------------------------+---+
|              17/03/2023|  3|
|              13/03/2023|  4|
|              13/03/2023|  5|
|              07/12/2022|  6|
|              06/12/2022|  7|
+------------------------+---+
only showing top 5 rows



monotonically_increasing_id() does not guarantee consecutive values. </br>
Using this to filter out the first two rows with condition like id > 2 does not ensure it is the lines that you want to be selected.

### With RDD

In [5]:
sc = spark.sparkContext
rdd = sc.textFile("./data/ext.csv")

# skip the first 4 lines
header = rdd.zipWithIndex().filter(lambda x: x[1] >= 5).keys()

split_lines = header.map(lambda x: x.split(";"))

Py4JJavaError: An error occurred while calling o43.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/wedivv/Code/others/pyspark/data/ext.csv
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:292)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:292)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:288)
	at org.apache.spark.api.java.JavaRDDLike.partitions(JavaRDDLike.

In [ ]:
split_lines.take(5)

[['03/04/2023', 'Pix enviado ', 'R E Rct Restaurantes Ltda', '-7,00'],
 ['03/04/2023', 'Pix enviado ', 'R E Rct Restaurantes Ltda', '-5,00']]

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("data_lancamento", StringType(), True),
    StructField("historico", StringType(), True),
    StructField("descricao", StringType(), True),
    StructField("valor", StringType(), True),
])

df = spark.createDataFrame(split_lines, schema)

df.show(5)


+---------------+------------+--------------------+-----+
|data_lancamento|   historico|           descricao|valor|
+---------------+------------+--------------------+-----+
|     03/04/2023|Pix enviado |R E Rct Restauran...|-7,00|
|     03/04/2023|Pix enviado |R E Rct Restauran...|-5,00|
+---------------+------------+--------------------+-----+
only showing top 2 rows



### In the real dataset I have one columns that doesn't have values.
so:

In [ ]:
sc = spark.sparkContext
rdd = sc.textFile("./data/Extrato.csv")

# skip the first 4 lines
header = rdd.zipWithIndex().filter(lambda x: x[1] >= 5).keys()

split_lines = header.map(lambda x: x.split(";"))

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("data_lancamento", StringType(), True),
    StructField("historico", StringType(), True),
    StructField("descricao", StringType(), True),
    StructField("valor", StringType(), True),
    StructField("saldo", StringType(), True)
])

# create the DataFrame with None for missing fields
df = split_lines.map(lambda x: tuple(x + [None]*(len(schema.fields)-len(x))))
df = spark.createDataFrame(df, schema)

df.show(5)


+---------------+------------+--------------------+-----+-----+
|data_lancamento|   historico|           descricao|valor|saldo|
+---------------+------------+--------------------+-----+-----+
|     03/04/2023|Pix enviado |R E Rct Restauran...|-7,00| null|
|     03/04/2023|Pix enviado |R E Rct Restauran...|-5,00| null|
+---------------+------------+--------------------+-----+-----+
only showing top 2 rows



DANGER! it will read everyline to apply the null's